In [2]:
os.sys.path.append('D:\\speyr\\Projets\\stephanie\\wagtail-realestate-github')

In [4]:
from mysite.settings.dev import *

In [8]:
INSTALLED_APPS

['home',
 'search',
 'blog',
 'realestate',
 'wagtail.contrib.modeladmin',
 'wagtailmenus',
 'wagtail.contrib.forms',
 'wagtail.contrib.redirects',
 'wagtail.embeds',
 'wagtail.sites',
 'wagtail.users',
 'wagtail.snippets',
 'wagtail.documents',
 'wagtail.images',
 'wagtail.search',
 'wagtail.admin',
 'wagtail.core',
 'wagtail.contrib.routable_page',
 'modelcluster',
 'taggit',
 'django.contrib.admin',
 'django.contrib.auth',
 'django.contrib.contenttypes',
 'django.contrib.sessions',
 'django.contrib.messages',
 'django.contrib.staticfiles',
 'django_extensions',
 'django.contrib.sites',
 'allauth',
 'allauth.account',
 'allauth.socialaccount']

In [5]:
import os
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
from wagtail.admin.auth import get_user_model
User = get_user_model()

AppRegistryNotReady: Apps aren't loaded yet.

In [6]:
import yaml

# from wagtail.
from django.contrib.auth.models import Group

AppRegistryNotReady: Apps aren't loaded yet.

In [5]:
group = Group(name='Customers')
group.save()

In [2]:
USERS_YAML = """
-
    user:
        username: admin
        email: admin@email.io
        first_name: Amin
        last_name: L'admin
        password: admin
    role: admin
-
    user:
        username: pduval
        email: pduval@email.io
        first_name: Pierre
        last_name: Duval
        password: pduval
    role: Editors
-
    user:
        username: jprevert
        email: jprevert@email.io
        first_name: Jacques
        last_name: Prévert
        password: jprevert
    role: Editors
-
    user:
        username: rdurand
        email: rdurand@email.io
        first_name: Robert
        last_name: Durand
        password: pduval
    role: Moderators
-
    user:
        username: jconrad
        email: jconrad@email.io
        first_name: Joseph
        last_name: Conrad
        password: jconrad
    role: Moderators
-
    user:
        username: gorwell
        email: gorwelld@email.io
        first_name: George
        last_name: Orwell
        password: gorwell
    role: Customers
"""


In [3]:
usersdata = yaml.safe_load(USERS_YAML)

for data in usersdata:
    role = data['role']
    if role == 'admin':
        print('admin')
    else:
        try:
            group = Group.objects.filter(name=role)[0]
            print('role: group:' + str(group))
            # print('type:' + str(type(group)))
        except IndexError:
            print('group {:s} not found'.format(role))
            print('group creating {:s}...'.format(role))
            group = Group(name=role)
            group.save()
            


In [ ]:
def delete_users(usersdata):
    usernames = [u['user']['username'] for u in usersdata]
    for username in usernames:
        q = User.objects.filter(username=username)
        if len(q) > 0:
            print('{:s} deleting...'.format(username))
            q[0].delete()
        else:
            print('{:s} not found '.format(username))

In [ ]:
def list_users():
    print(get_user_model().objects.all())

In [ ]:
def populate_users(usersdata, delete=False):
    userclass = get_user_model()

    for data in usersdata:
        role = data['role']
        userdata = data['user']
        username = userdata['username']
        q = userclass.objects.filter(username=username)
        if len(q) > 0:
            # remove any user with that name
            if delete:
                try:
                    q[0].delete()
                except IndexError:
                    pass

        if role == 'admin':
            user = userclass.objects.create_superuser(**userdata)
            user.save()
        else:
            group = None
            try:
                group = Group.objects.filter(name=role)[0]
            except IndexError:
                print('group {:s} not found creating...'.format(role))
                group = Group.objects.create(name=role)

            user = userclass.objects.create_user(**userdata)
            if group:
                user.groups.add(group)
            user.save()


In [ ]:
list_users()
print(Group.objects.all())

In [ ]:
User.objects.all()

In [ ]:
# delete_users(usersdata)

In [ ]:
populate_users(usersdata)

In [ ]:
list_users()